In [ ]:
# Iran Monte Carlo Simulation on Brent Crude Options

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Load Brent options data
df = pd.read_csv("../data/brent_options_data.csv")
df = df.dropna(subset=["impliedVolatility"])

# Filter OTM call options
df = df[(df["inTheMoney"] == False) & (df["strike"] > 75)]
df = df.head(50).copy()

# Assumptions
S = 83.0  # Brent price
r = 0.05  # risk-free rate
simulations = 10000

# Time to expiration in years
def get_T(expiration_date):
    exp_date = datetime.strptime(expiration_date, "%Y-%m-%d")
    delta = (exp_date - datetime.today()).days
    return max(delta / 365, 1 / 365)

df["T"] = df["expirationDate"].apply(get_T)

# Monte Carlo Simulation
def monte_carlo(S, K, T, r, sigma, option_type="call", simulations=10000):
    Z = np.random.standard_normal(simulations)
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    if option_type == "call":
        payoff = np.maximum(ST - K, 0)
    elif option_type == "put":
        payoff = np.maximum(K - ST, 0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

